In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

c:\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
LR = 1e-3
env = gym.make('MountainCar-v0')
env.reset()
goal_steps = 50
score_requirement = 50
initial_game = 10

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
def some_random_games_first():
  for episode in range(5):
    env.reset()
    for t in range(200):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print(info)
        if done:
            break
      
      
      
      
#some_random_games_first()


In [15]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_game):
        score =0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0,3)
            #action = 0 or 1 or 2 
            observation, reward, done, info = env.step(action)
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation)>0:
                #game_memory save all obersation and action
                game_memory.append([prev_observation, action])
                #print('prev_observation:',prev_observation,' action:',action)
            prev_observation = observation
            score +=reward
            if done:
                break
        #if score >= score_requirement:
            #accepted_scores just save score above score_requirement
            accepted_scores.append(score)
            #to change action '1'-->[0,1] or action '0' -->[1,0] for suit with numpy data style
            # we change to [0,1] because we also keep 0 or 1, but some game have more action, so we use [0,1]
            for data in game_memory:
                if data[1]== 0:
                    output = [0,0]
                if data[1] == 1:
                    output = [1,0]
                elif data[1] == 2:
                    ouput = [1,1]
                #saving our traning data
                return training_data.append([data[0],output])
        env.reset()
        #save all score values of each espiso, also counting value not enough for the requirement
        scores.append(score)
    #just in case you wanted to reference later
    training_data_save = np.array(training_data)
    print(training_data_save.shape)
    np.save('./saved.npy', training_data_save)
    
    #some stats here,to further illustrate the neural network magic
    print('Average accepted score:', mean(accepted_scores))
    print('Median score for accepted scores:', median(accepted_scores))
    #counter is keep track how many value repeat
    print(Counter(accepted_scores))
    return training_data
initial_population()

In [16]:
hihi = initial_population()
type(hihi)

NoneType

In [6]:
def neural_network_model(input_size):
    #1 in input_size mean just observation source for input 
    network = input_data(shape=[None, input_size, 1], name='input')
    #128 nodes for fisrt layer, each node have activation is relu for linear function
    network = fully_connected(network, 128, activation='relu')
    #0.8 is keep rate
    network = dropout(network, 0.8)
    #seconde layer
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')
    #sometimes have out of memony, so just change number of layer
    return model


In [7]:
def train_model(training_data, model=False):
    #training_data have pype [[4 observation][0,1 or 1,0]]
    #[i[0] for i in training_data] mean all element in first place (4 observations)
    # X need np.array because training_data has type is list, can't not reshape -->should be convert to numpy.array type
    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)#---> X has shape: (totall number,4,1) -->the official shape for training
    #[i[0] for i in training_data] mean all elements in the secon place [0,1] or [1,0]
    y = [i[1] for i in training_data] #----> [0,1] or [1,0]
    #print(X.shape)
    #print(y[0])
    if not model:
        #run neural_network_model
        model = neural_network_model(input_size = len(X[0])) #len(X[0])=4
    #just run 1 time becasue model.fit can re-run
    model.fit({'input': X}, {'targets': y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id='openai_learning')
    #n-epoch in simple case don't too high, <=95% accurate
    #snapshot_step mean how much step trained will print out the screen
    return model
training_data = initial_population()
model = train_model(training_data)

UnboundLocalError: local variable 'output' referenced before assignment